# Pydantic fundamentals

### without pydantic

In [6]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name = "John", age = 37, gender = "male")
person1

In [3]:
person1.name, person1.age

('John', 37)

In [7]:
#this works in Python because it is not hard typed language
person2 = Person(name= 3234+244, gender = True, age = "minus ten")
person2

#although nice, this freedom is a problem in validation and robustness.
#this is where Pyndantic is needed

### validation of the Person class

In [13]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        if not isinstance(name, str):
            raise TypeError(f"must be of type str, not {type(name)}")
        self.gender = gender
        self.age = age

try:
    person1 = Person(name = 4357653, age = 37, gender = "male")
except TypeError as err:
    print(err)

must be of type str, not <class 'int'>


In [25]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        if not isinstance(name, str):
            raise TypeError(f"must be of type str, not {type(name)}")
        self.gender = gender
        if not isinstance(age, int):
            raise TypeError(f"age must be of type in not ({type(age)})")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124, not {age}")
        self.age = age

    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"

try:
    person3 = Person(name=455, age=-54, gender="M") 
    
except TypeError as err:
    print(f"Caught Type Error: {err}") 
    
except ValueError as err:
    print(f"Caught Value Error: {err}")

Caught Type Error: must be of type str, not <class 'int'>


In [26]:
person4 = Person(name="Bella", age=54, gender="F") 
person4

Person(Bella, F, 54)

In [27]:
# this works becuase the validation is only done when it is instanciated.
person4.age = -4
person4

Person(Bella, F, -4)

In [29]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        if not isinstance(name, str):
            raise TypeError(f"must be of type str, not {type(name)}")
        self.gender = gender
        self.age = age

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if not isinstance(age, int):
            raise TypeError(f"age must be of type in not ({type(age)})")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124, not {age}")
        self._age = age

    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"

person5 = Person(name="Leo", age=4, gender="M") 
person5

Person(Leo, M, 4)

In [31]:
try:
    person5.age = -4
except ValueError as err:
    print(err)

age must be between 0 and 124, not -4


In [32]:
try:
    Person(name=445, age=4, gender="M") 
except ValueError as err:
    print(err)

TypeError: must be of type str, not <class 'int'>

## Validate using Pydantic

In [50]:
from pydantic import BaseModel
#inherents from BaseModel 

class Person_py(BaseModel):
    name: str
    gender: str
    age: int

person6= Person_py(name= "Christina", gender="F", age=29)
person6

Person_py(name='Christina', gender='F', age=29)

In [51]:
person6.age = 36
person6

Person_py(name='Christina', gender='F', age=36)

In [52]:
#issue
person6.age = "thirty"
person6

Person_py(name='Christina', gender='F', age='thirty')

In [54]:
from pydantic import ValidationError

try:
    Person_py(name= 244242, gender="F", age=29)
except ValidationError as err:
    print(err)

1 validation error for Person_py
name
  Input should be a valid string [type=string_type, input_value=244242, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [55]:
# str "29" is coerced in 29
Person_py(name= "Nina", gender="F", age="25")

Person_py(name='Nina', gender='F', age=25)

In [59]:
from pydantic import ConfigDict
class Person_py_correct(BaseModel):
    name: str
    gender: str
    age: int
    
    model_config = ConfigDict(validate_assignment=True)

person7= Person_py_correct(name= "Christina", gender="F", age="29")

try:
    person7.age = "thirty"
except ValidationError as err:
    print(err)


person7.age = 10
person7

1 validation error for Person_py_correct
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person_py_correct(name='Christina', gender='F', age=10)

In [61]:
try:
    Person_py_correct(name = 3.14555, gender=2.744, age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person_py_correct
name
  Input should be a valid string [type=string_type, input_value=3.14555, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.744, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [63]:
try:
    Person_py_correct(name = "Sofia", gender="F", age = -4)
except ValidationError as err:
    print(err)

### Add age validation

In [66]:
from pydantic import Field
from typing import Literal

class Person_py_correct_age(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt =-1, lt = 125)
    
    model_config = ConfigDict(validate_assignment=True)


try:
    Person_py_correct_age(name= "Christina", gender="Fem", age=-5)
except ValidationError as err:
    print(err)

2 validation errors for Person_py_correct_age
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


### Serilalisation and desentrilation

We go from OOP instances to json and from json to OOP instance !
needed for API and send with json and then deserialize agian back to Pythion object.

In [68]:
#serialization
person7.model_dump()

{'name': 'Christina', 'gender': 'F', 'age': 10}

In [ ]:
#serialization
import json
with open("person.json", "w") as file:
    json.dump(person7.model_dump(), file)

In [75]:
with open("person.json", "r") as file:
    json_data = file.read()


#json_data
#print(json_data)
print(repr(json_data))

#deserialization
Person_py_correct_age.model_validate_json(json_data)


'{"name": "Christina", "gender": "F", "age": 10}'


Person_py_correct_age(name='Christina', gender='F', age=10)